# PIERS Data Cleaning

This notebook addresses missing data and other issues in the PIERS BOL data. This work is predominantly exploratory in nature, and some of these processes may eventually move to the ETL step. 

In [1]:
import pandas as pd #v2.1.3
import polars as pl #v0.20.7
import plotly_express as px #v0.4.1 
import missingno as msno #v0.5.2
import datetime as dt
from polars.testing import assert_frame_equal


#display settings
pd.set_option('display.max_columns', None)

#enable string cache for polars categoricals
pl.enable_string_cache()

Convenience steps:
- drop unused columns
- create year and month (e.g., 202305) columns 
- replace zero values with nulls in volume columns
- create lane_id and lane_name columns
- create voyage_id from year, vessel IMO code, voyage number, and origin/destination country 
    - this method turns out to be problematic, due to multiple voyage numbers occuring within vessel-date-port groups.
- add direction column showing 'import' or 'export' to allow easy merging
- get most commonly used SCAC and name for each carrier

In [2]:
#set paths
imports_path = 'data/raw_parquet/imports/'
exports_path = 'data/raw_parquet/exports/'
#init lazy dataframes
imports_lf = (
    pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
    .select(
        'weight',
        'weight_unit',
        'qty',
        'qty_type',
        'teus',
        #'value_est',
        'date_arrival',
        'container_piece_count',
        'commod_short_desc_qty',
        'origin_territory',
        'origin_region',
        'arrival_port_code',
        'arrival_port_name',
        'departure_port_code',
        'departure_port_name',
        #'dest_final',
        'coast_region',
        'clearing_district',
        #'place_receipt',
        #'shipper_name',
        #'shipper_address',
        #'consignee_name',
        #'consignee_address',
        #'notify_party1_name',
        #'notify_party1_address',
        #'notify_party2_name',
        #'notify_party2_address',
        'commod_desc_raw',
        'container_id_marks',
        'marks_desc',
        'hs_code',
        'joc_code',
        'commod_short_desc',
        'container_ids',
        'carrier_name',
        'carrier_scac',
        'vessel_name',
        'voyage_number',
        #'precarrier',
        'vessel_id',
        #'inbond_code',
        #'transport_mode',
        'bol_number',
        'bol_id'
    )
    .with_columns([
        #create year and month columns
        pl.col('date_arrival').dt.year().alias('year'),
        pl.col('date_arrival').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('import').cast(pl.Categorical).alias('direction'), 
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name for each scac and vise-versa to normalize changes in names and codes 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('origin_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)
exports_lf = (
    pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')
    #drop unused columns
    .select(
        #'shipper',
        #'shipper_address',
        'weight',
        'weight_unit',
        'qty',
        'quantity_type',
        'teus',
        'carrier_name',
        'carrier_scac',
        'vessel_name',
        'voyage_number',
        'bol_number',
        'vessel_id',
        #'value_est',
        'departure_port_code',
        'departure_port_name',
        'container_ids',
        'container_piece_count',
        'coast_region',
        'commod_desc_raw',
        'commod_short_desc',
        'hs_code',
        'joc_code',
        'commod_short_desc_qty',
        'date_departure',
        #'origin',
        'dest_territory',
        'dest_region',
        'arrival_port_code',
        'arrival_port_name',
        'bol_id'
    )
    .with_columns([
        #create year and month columns
        pl.col('date_departure').dt.year().alias('year'),
        pl.col('date_departure').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('export').cast(pl.Categorical).alias('direction'),
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name and scac 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('dest_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)

In [3]:
imports_lf.schema

OrderedDict([('weight', Float64),
             ('weight_unit', Categorical(ordering='physical')),
             ('qty', Float64),
             ('qty_type', Categorical(ordering='physical')),
             ('teus', Float64),
             ('date_arrival', Datetime(time_unit='us', time_zone=None)),
             ('container_piece_count', Int32),
             ('commod_short_desc_qty', String),
             ('origin_territory', Categorical(ordering='physical')),
             ('origin_region', Categorical(ordering='physical')),
             ('arrival_port_code', Categorical(ordering='physical')),
             ('arrival_port_name', Categorical(ordering='physical')),
             ('departure_port_code', Categorical(ordering='physical')),
             ('departure_port_name', Categorical(ordering='physical')),
             ('coast_region', Categorical(ordering='physical')),
             ('clearing_district', Categorical(ordering='physical')),
             ('commod_desc_raw', String),
             (

In [4]:
exports_lf.schema

OrderedDict([('weight', Float64),
             ('weight_unit', Categorical(ordering='physical')),
             ('qty', Float64),
             ('quantity_type', Categorical(ordering='physical')),
             ('teus', Float64),
             ('carrier_name', Categorical(ordering='physical')),
             ('carrier_scac', Categorical(ordering='physical')),
             ('vessel_name', String),
             ('voyage_number', String),
             ('bol_number', String),
             ('vessel_id', Int32),
             ('departure_port_code', Categorical(ordering='physical')),
             ('departure_port_name', Categorical(ordering='physical')),
             ('container_ids', String),
             ('container_piece_count', Int32),
             ('coast_region', Categorical(ordering='physical')),
             ('commod_desc_raw', String),
             ('commod_short_desc', String),
             ('hs_code', String),
             ('joc_code', String),
             ('commod_short_desc_qty', Str

## Notebook functions

In [5]:
#fill nulls in volume cols with mean
def fill_volume(lf):
    '''ad hod function to fill volume columns with their means'''
    return (
        lf
        .with_columns([
            pl.col('teus').replace(0,None).fill_null(strategy='mean'),
            pl.col('weight').replace(0,None).fill_null(strategy='mean'),
            pl.col('qty').replace(0,None).fill_null(strategy='mean')
            ])
        )

#plotly graph inspecting nulls over time by group
def nulls_over_time_plotly(data_lf, group_var, time_var, value_var, title=False):
    '''
    Plots proportion of null values over time by group.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        time_var - str - the name of the time column (e.g., year, month) over which values will be counted
        value_var - str - the name of the column containing the variable in question
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    df = (
        #select relevant columns
        data_lf.select([group_var, time_var, value_var])
        #group by, creating null count and non-null count cols
        .group_by(group_var, time_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        #compute percent null and fill new column
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort(time_var, group_var)
    ).collect()
    #plot
    fig = px.line(
        data_frame=df,
        x=time_var, y='null_percent',
        color=group_var,
        title= 'Count of nulls over time by source frame.' if not title else title
    )
    fig.show()
    del df

#plotly graph inspecting nulls by group
def nulls_by_group_plotly(data_lf, group_var, value_var, title=False):
    '''Plots proportion of null values in the given groups'''
    df = (
        data_lf.select([group_var, value_var])
        .group_by(group_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        .cast({group_var:pl.Utf8})
        .sort('null_percent', descending=True)
    ).collect()
    #plot
    fig = px.bar(
        data_frame=df,
        x=group_var, y='null_percent',
        title= 'Null percent by group.' if not title else title
    )
    fig.show()

#fill nulls over groups given a single unique value per group
def fill_nulls_by_group(data_lf, group_vars, val_var):
    '''Fills null values by group if and only if the val_var for that group contains exactly one non-null unique value.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_vars - iterable - the names of the columns by which groups will be created
        val_var - string - the name of the column in which nulls will be filled
    OUTPUT:
        filled_lf - the resultant lazyframe 
    DEPENDS ON:
        polars - current version written in polars 0.20.1
    '''
    filled_lf = (
        data_lf.with_columns(
            #if the group contains exactly one unique value: 
            pl.when(pl.col(val_var).drop_nulls().unique(maintain_order=True).len().over(group_vars)==1)
            #then fill the group with that value
            .then(pl.col(val_var).fill_null(pl.col(val_var).drop_nulls().unique(maintain_order=True).first().over(group_vars)))
            #otherwise do nothing
            .otherwise(pl.col(val_var))
            )
        )
    return filled_lf

def is_one2one(lf, col1, col2):
    '''checks if the two pl.LazyFrame columns constitute a 1-1 pairing'''
    forward = lf.group_by(col1).agg(pl.col(col2).drop_nulls().n_unique()).select(col2).max().collect().item() == 1
    back = lf.group_by(col2).agg(pl.col(col1).drop_nulls().n_unique()).select(col1).max().collect().item() == 1
    return (forward and back)

def count_unique_by_group(lf, group_vars, val_var):
    '''returns a dataframe of unique observations for the value variable over each group'''
    df = (
        lf.group_by(group_vars)
        .agg(
            pl.col(val_var).unique().alias('unique_values'),
            pl.col(val_var).n_unique().alias('n_unique')
        )
        .drop_nulls()
        .sort('n_unique', descending=True)
        .collect()
    )
    return df

def add_primary_carrier(lf):
    '''alternative ad hoc function to find primary carrier for each vessel and indicate cargo sharing'''
    lf = (
        lf.with_columns(
            pl.col('teus').sum()
            .over('vessel_id', 'month', 'unified_carrier_scac')
            .alias('sum_teus')
            )
        .with_columns(
            pl.col('unified_carrier_scac')
            .sort_by('sum_teus', descending=True)
            .drop_nulls().first()
            .over('vessel_id', 'month')
            .alias('vessel_owner')
            )
        #add bool col if bol is from primary carrier
        .with_columns(
            (pl.col('unified_carrier_scac')==pl.col('vessel_owner'))
            .alias('primary_cargo')
            )
    )
    return lf

def sharing_over_time_plotly(data_lf, group_var, include_missing_voyages=True, limit=10, title=False):
    '''
    Plots proportion of shared cargo over time (months) by group_var.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        include_missing_voyages - bool - default=True, when False, drops missing voyages
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    if not include_missing_voyages:
        df = data_lf.drop_nulls('voyage_id')
    else:
        df = data_lf
    
    df = (
        #select relevant columns
        df.select([group_var, 'month', 'primary_cargo', 'teus'])
        #sum teus over each group-month-shared 
        .group_by(group_var, 'month')
        .agg(
            (pl.col('teus')*pl.col('primary_cargo')).sum().alias('total_primary'),
            pl.col('teus').sum().alias('total_teus')
        )
        #create proportion shared
        .with_columns((1-(pl.col('total_primary')/pl.col('total_teus'))).alias('prop_shared'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort('month')
    ).collect()

    #limit categories
    top_groups = (
        data_lf.group_by(group_var)
        .agg(pl.col('teus').sum())
        .sort('teus', descending=True)
        .select(group_var)
        .limit(limit)
        .collect()
        .to_series()
        .cast(pl.Utf8)
    )
    
    #plot
    fig = px.line(
        data_frame=df.filter(pl.col(group_var).is_in(top_groups)).with_columns(pl.col('month').str.to_datetime('%Y%m')),
        x='month', y='prop_shared',
        color=group_var,
        title= 'Proportion of shared cargo over time.' if not title else title,
        labels={
            'prop_shared':'Proportion of cargo from non-primary carrier',
            'month':'Month'
        }
    )
    fig.show()


## Missing Data


In [6]:
%%script echo skipping
print('Missing Values Matrix for Exports. \nThe vertical axis corresponds to date, with earliest at the top.  \nNote the daily aggregation counts the row as non-null if any value that day is non-null.')
msno.matrix(
    exports_lf
    #convert all columns except date to boolean False=null
    .with_columns(pl.all().exclude('date_departure').is_not_null())
    #aggregate by day to stay within visualization limitations 
    .group_by('date_departure').sum()
    .sort('date_departure')
    #convert False values back to null 
    .with_columns(pl.all().replace(0,None))
    .collect()
    .to_pandas()
    );

skipping


In [7]:
%%script echo skipping
print('Missing Values Matrix for Imports. \nThe vertical axis corresponds to date, with earliest at the top. \nNote the daily aggregation counts the row as non-null if any value that day is non-null.')
msno.matrix(
    imports_lf
    #convert all columns except date to boolean False=null
    .with_columns(pl.all().exclude('date_arrival').is_not_null())
    #aggregate by day to stay within visualization limitations 
    .group_by('date_arrival').sum()
    .sort('date_arrival')
    #convert False values back to null 
    .with_columns(pl.all().replace(0,None))
    .collect()
    .to_pandas()
    );

skipping


### Zero values in TEU and other volume columns

For the time being, we fill the zero (i.e., missing) volume data with the mean from non-zero observations. This step is expected to change as beter metadata is gathered. 

In [8]:
imports_lf = fill_volume(imports_lf)
exports_lf = fill_volume(exports_lf)    

### Missing Vessel and Voyage IDs

Identifying unique vessels and voyages is critical to studying how long each vessel remains in port, how often carriers utilize alliances (e.g., what proportion of a vessel's cargo is from another carrier), and other quality metrics. The issue is especially concerning given the high proportion (~30% in some time periods) of missing data, as can be seen below.

In [9]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'vessel_id'),
    exports_lf.select('direction', 'month', 'vessel_id')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='vessel_id',
    title='Proportion of Missing Vessel IDs over time (original data).')

In [10]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'vessel_name'),
    exports_lf.select('direction', 'month', 'vessel_name')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='vessel_name',
    title='Proportion of Missing Vessel Names over time (original data).')

In [11]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'voyage_id'),
    exports_lf.select('direction', 'month', 'voyage_id')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='voyage_id',
    title='Proportion of Missing Voyage IDs over time (original data).')

## Primary Carrier

For any given voyage, we assign primary carrier based on the proportion of TEUs from each carrier imported/exported on that voyage (max proportion is primary carrier). 

In [12]:
#add primary carrier and shared cargo columns to lfs
imports_lf = add_primary_carrier(imports_lf)
exports_lf = add_primary_carrier(exports_lf)

In [13]:
lf = pl.concat([imports_lf,exports_lf], how='diagonal')

sharing_over_time_plotly(
    data_lf=lf,
    group_var='direction')

In [14]:
lf = pl.concat([imports_lf,exports_lf], how='diagonal')

sharing_over_time_plotly(
    data_lf=lf,
    include_missing_voyages=False,
    group_var='direction')

In [15]:
sharing_over_time_plotly(
    data_lf=exports_lf,
    group_var='departure_port_name',
    title='Proportion of shared cargo over time (Top 10 export ports by total TEUs).')

In [16]:
sharing_over_time_plotly(
    data_lf=exports_lf,
    group_var='departure_port_code',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time (Top 10 export ports by total TEUs).')

In [17]:
sharing_over_time_plotly(
    data_lf=exports_lf,
    group_var='lane_name',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time (Top 10 export lanes by total TEUs).')

In [18]:
sharing_over_time_plotly(
    data_lf=imports_lf,
    group_var='lane_name',
    title='Proportion of shared cargo over time (Top 10 import lanes by total TEUs).')

In [20]:
sharing_over_time_plotly(
    data_lf=imports_lf,
    group_var='vessel_owner',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time (Top 10 Vessel Owners by total TEUs imported).')

In [21]:
sharing_over_time_plotly(
    data_lf=exports_lf,
    group_var='vessel_owner',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time (Top 10 Vessel Owners by total TEUs exported).')

In [24]:
#filter by MSC and Maersk controlled voyages
lf = (
    imports_lf.filter(
        pl.col('vessel_owner').is_in(['MEDU','MAEU']),
        pl.col('unified_carrier_scac').is_in(['MEDU','MAEU'])
    )
)
sharing_over_time_plotly(
    data_lf=lf,
    group_var='lane_name',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time for MSC and Maersk vessels (Top 10 import lanes by total TEUs).')


In [25]:
#filter by MSC and Maersk controlled voyages
lf = (
    exports_lf.filter(
        pl.col('vessel_owner').is_in(['MSCU','MLSL']),
        pl.col('unified_carrier_scac').is_in(['MSCU','MLSL'])
    )
)
sharing_over_time_plotly(
    data_lf=lf,
    group_var='lane_name',
    include_missing_voyages=False,
    title='Proportion of shared cargo over time for MSC and Maersk vessels (Top 10 export lanes by total TEUs).')

### 'BULK' Carrier SCAC

Many bols show "BULK" in place of carrier code, and at least some bols also show legitimate carrier names. We'll need to clean this up. Perhaps assign carrier code based on carrier name where possible and null values for the reminder?

## Unpacking Lists (commodity codes, etc)

List-like columns include:
- container_id_marks
- marks_desc
- hs_code
- job_code
- commod_desc_raw
- container_ids
- commod_short_desc_qty
- 

In [ ]:
imports_lf.schema

In [ ]:
df = imports_lf.select('bol_id', 'teus', 'date_arrival', 'arrival_port_name', 'lane_name', 'container_id_marks', 'marks_desc', 'container_ids', 'hs_code').collect()

In [ ]:
df2 = (
    df.with_columns(pl.col('hs_code').str.split(by=' '))
    .filter(pl.col('hs_code').list.len()>1)
    .sort('bol_id')
)

df2.head()

In [ ]:
df_long = df2.explode('hs_code')
df_long.head()

In [ ]:
df_long.with_columns(pl.col('hs_code').cast(pl.Utf8).str.split(by='/')).filter(pl.col('hs_code').list.len()>1)

## Export Clean data to parquet
